In [1]:
import csv
import numpy as np
import pandas as pd
import pathlib
import os
from collections import defaultdict

In [2]:

def createAnswerKey(fileName):
    # open AnswerKey csv file and create a dictionary of all the answers, where the format is (Lecture#, question#): Answer
    questionList = []
    answers = {}
    lectureNum = ""
    with open(fileName) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                questionList = row
                line_count+=1
            else:
                for answerIndex in range(len(row)):
                    if answerIndex > 0:
                        lectureNum = row[0]
                        answer = row[answerIndex]
                        answers[(lectureNum,questionList[answerIndex])] = answer

                line_count+=1

    #print(answers)
    return questionList, answers, lectureNum
                

In [3]:
def cleanAnswer(studAnswer, key):
    try:
        studAnswer = int(float(studAnswer))
        if 'False' in key or 'True' in key:
            if studAnswer == 0:
                return 'False'
            else:
                return 'True'
        else:
            return int(float(studAnswer))
    except:
        return studAnswer

In [4]:
def partAnalysis(fileName, questionList, answers, lectureNum):
   #print('reading file', fileName)
    response = pd.read_csv(fileName)
    analysis = []
    allEmails = response['Email Address'].values
    answeredQuestions = []
    
    for question in response:
        if question in questionList:
            answeredQuestions.append(question)
            
            countCorrect = 0
            countTotal = 0

            studentSubmissions = {}
            index = 0
            
            #iterate through all the student responses for the question
            for studAnswer in response[question]:  
                
                #check if valid/non-null submission
                if not pd.isnull(studAnswer):
                    
                    #assign student to their email
                    student = allEmails[index]
                    
                    #check if this is the student's first valid submission 
                    if student not in studentSubmissions:
                        studentSubmissions[student] = studAnswer

                        #used 'in' instead of '==' to account for whitespace in answers
                        #try except block to check compare float responses vs integer answers
                        
                        #print(studAnswer)
                        studAnswer = cleanAnswer(studAnswer,answers[(lectureNum, question)])
                        #print(studAnswer)
                        if str(studAnswer) in answers[(lectureNum, question)]:
                            
            
                            countCorrect+=1
                            #print("Answer:", answers[(lectureNum, question)])
                            #print(str(studAnswer))
            
    

                        countTotal +=1

                index+=1

            if countTotal > 0:
                #print(countCorrect,"/",countTotal)

                analysis.append(float(countCorrect/countTotal))
                

    return analysis, answeredQuestions
            
        
   


In [9]:
def listToDict(fullLectureAnalysis, qList):
    #print(qList)
    lectureDict = defaultdict(list)
    for i in range(4):
        for q in range(len(qList)):
            lectureDict[qList[q]].append(fullLectureAnalysis[i][q])
            
    return lectureDict

In [10]:
answerList = []

for path in pathlib.Path("./ANSWERS").iterdir():
    if path.is_file():
        answerList.append(str(path))

for ans in answerList:
    qList, answers, lecNum = createAnswerKey(ans)
    print(f"------------- Starting {lecNum} -------------")
    lec = lecNum.replace(" ", "").lower()
    #print(lec)
    for path in pathlib.Path("./RESPONSES").iterdir():
        if path.is_dir() and lec in str(path):
            
            
            list_of_files = sorted( filter( lambda x: os.path.isfile(os.path.join(path, x)),
                        os.listdir(path) ) )
            #print(list_of_files)
            fullLectureAnalysis = []
            
            for file in list_of_files:
                analysis, answeredQuestions = partAnalysis(f'{path}/{file}', qList, answers, lecNum)
                #print(analysis)
                fullLectureAnalysis.append(analysis)
                #print("---------")
                
            
            lectureDict = listToDict(fullLectureAnalysis, answeredQuestions) 
            #print(lectureDict)
            df = pd.DataFrame.from_dict(lectureDict, orient = 'index', columns = ['A00', 'B00', 'C00', 'D00'])
            print(df)
            #df.to_csv(f'./RESULTS/{lecNum}.csv')
            #print(f"------------- OUTPUT {lecNum} TO CSV -------------")
                
            
                
#             for subPath in path.iterdir():
#                 if subPath.is_file():
#                     analysis = partAnalysis(subPath, qList, answers, lecNum)
# #                     for key, val in analysis.items():
# #                         print(val)
#                     print(analysis)
#                     print("---------")


    print()


------------- Starting Lecture 13 -------------
                                                         A00       B00  \
Images                                              0.000000  0.000000   
Tuples                                              0.563830  0.533333   
Images as 2d List of Tuples                         0.738462  0.850467   
Working with Real Images using the CSE8AImage L...  0.590164  0.646465   

                                                         C00       D00  
Images                                              0.000000  0.000000  
Tuples                                              0.686047  0.637500  
Images as 2d List of Tuples                         0.839286  0.769231  
Working with Real Images using the CSE8AImage L...  0.644444  0.670213  

------------- Starting Lecture 12 -------------
        A00       B00       C00       D00
1  0.577778  0.350515  0.671429  0.761905
2  0.607477  0.612245  0.787234  0.765432
3  0.535211  0.537634  0.448276  0.540984

                      A00       B00       C00       D00
Review Exercise  0.564815  0.378151  0.803922  0.773810
Big idea #1      0.370370  0.145299  0.587629  0.654762
Big idea #2      0.457143  0.447368  0.628866  0.658537
Big idea #3      0.550725  0.500000  0.831579  0.831325
Big idea #4      0.357143  0.507463  0.531250  0.734940
Big idea #5      0.444444  0.531250  0.488372  0.291667

------------- Starting Lecture 18 -------------
                      A00       B00       C00       D00
CSV              0.843373  0.609756  0.675000  0.776471
CSV.1            0.527273  0.620000  0.760000  0.693182
List of dicts    0.698795  0.708333  0.545455  0.488636
List of dicts.1  0.782051  0.762500  0.416667  0.297619
List of dicts.2  0.385542  0.340206  0.484848  0.525000
Dict of dicts    0.413333  0.472527  0.333333  0.438356
Dict of dicts.1  0.418919  0.417582  0.387755  0.428571
Dict of dicts.2  0.460000  0.680851  0.782609  0.716418

------------- Starting Lecture 20 -------------
      

In [ ]:
#### TEST
qList, answers, lecNum = createAnswerKey('./ANSWERS/Lecture Answers - lect3.csv')
print(answers)

print("-----------------QUESTIONS-----------------")
# for q in qList:
#     print(q)
    
#path = "./TESTING/testDuplicates.csv"
path = "./RESPONSES/lecture3/Lecture 3 Participation Form - A00 (Responses).csv"

analysis, answeredQuestions = partAnalysis(path, qList, answers, lecNum)
print(answeredQuestions)

print(analysis)

#print(analysis)
#analysis.to_csv(f'./TESTING/TestB00.csv')
            
